# 개발환경 구축

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta
import re
import time

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # FutureWarning 제거

In [ ]:
# Selenium 설치
!pip install selenium
!pip install webdriver_manager


# Chrome WebDriver 설치
!apt-get update
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.2/400.2 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.5 MB/s eta 0:00:00
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [458 kB]
Hit:10 https://ppa.launchpadcontent.ne

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

service = Service()

# Chrome WebDriver 실행
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # 화면 출력 없이 실행하려면 이 옵션을 사용
options.add_argument('--no-sandbox')  # Colab 환경에서 실행할 때 필요한 옵션
options.add_argument('--disable-dev-shm-usage')  # Colab 환경에서 실행할 때 필요한 옵션
driver = webdriver.Chrome(service=service, options=options)  # 'chromedriver'는 PATH에 있는 경우 생략 가능

# 버전 확인
print("Chrome WebDriver Version:", driver.capabilities['chrome']['chromedriverVersion'])


# WebDriver 종료
driver.quit()


Chrome WebDriver Version: 116.0.5845.96 (1a391816688002153ef791ffe60d9e899a71a037-refs/branch-heads/5845@{#1382})


In [ ]:
!pip install random_user_agent

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 15.6 MB/s eta 0:00:00


In [ ]:
from random_user_agent.user_agent import UserAgent
from random_user_agent.params import SoftwareName, OperatingSystem


class RandomUserAgentTest:
    def __init__(self):
        self.get_headers()
        print(self.headers)

    def set_user_agent(self):
        software_names = [SoftwareName.CHROME.value]
        operating_systems = [OperatingSystem.WINDOWS.value, OperatingSystem.LINUX.value]
        user_agent_rotator = UserAgent(software_names=software_names, operating_systems=operating_systems, limit=100)
        self.user_agent = user_agent_rotator.get_random_user_agent()

    def get_headers(self):
        self.set_user_agent()
        self.headers = {
            "User-Agent": self.user_agent,
        }


if __name__ == "__main__":
    test = RandomUserAgentTest()

{'User-Agent': '24.0.1284.0.0 (Windows NT 5.1) AppleWebKit/534.0 (KHTML, like Gecko) Chrome/24.0.1284.0.3.742.3 Safari/534.3'}


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#아하

**알고리즘**

```
# 먼저 질문 몇 개인지 파악하기
# 질문갯수//10 번 for문 돌리기
# 그만큼 버튼 클릭한 다음에 링크+제목+날짜 한꺼번에 수집
# 링크 들어가서 답변 내용 가져오기
# 답변에서 첫 두 문장 삭제 (안녕하세요+저는~입니다.)
```



beautifulsoup으로 질문 개수 파악

In [ ]:
# 크롤링 링크 (한 페이지)
link="https://www.a-ha.io/search/%EA%B3%B5%EB%AA%A8%EC%A3%BC"

# html태그 가져오기
response=requests.get(link)
soup=BeautifulSoup(response.text,'html.parser')

# 현재 페이지의 총 질문 개수 파악하기
q=soup.find_all('span',{'class':'tw-ml-2'})
question=q[0].text
q_num = int(re.sub(r'[^0-9]', '', question))
print(q_num)

303


In [ ]:
# 버튼을 눌러야하는 횟수 계산
click_num=(q_num-1)//10

Selenium으로 버튼 클릭 -> 질문 링크 가져오기

In [ ]:
# 맨 처음 페이지 버튼 xpath
target_present = EC.presence_of_element_located((By.XPATH, '//*[@id="__layout"]/div/div[1]/div[2]/section[3]/div/div/div/div[11]/button'))
# 크롤링 링크
link = "https://www.a-ha.io/search/%EA%B3%B5%EB%AA%A8%EC%A3%BC"


try:
  driver.get(link)
  driver.implicitly_wait(10)
  element=WebDriverWait(driver, 10).until(target_present) # 버튼 인식할 때까지 기다림
  driver.implicitly_wait(10)
  backtrans = element.text # 버튼 문구

  if (backtrans=='')|(backtrans==' '): # 아직 로딩이 안 됨
    element=WebDriverWait(driver, 10).until(target_present)
    backtrans = element.text
    print(0,backtrans)
  else:
    print(1,backtrans)

  # click_num만큼 버튼 연쇄적으로 클릭
  for i in range(2,click_num+2):
    element.click()
    time.sleep(2)
    # 버튼 xpath
    dk =  EC.presence_of_element_located((By.XPATH, '//*[@id="__layout"]/div/div[1]/div[2]/section[3]/div/div/div/div['+str(i)+'1]/button'))
    element=WebDriverWait(driver, 10).until(dk)
    time.sleep(0.2)
    print(i)

  backtrans = element.text
  if (backtrans=='')|(backtrans==' '):
    element=WebDriverWait(driver, 10).until(target_present)
    backtrans = element.text
    print(0,backtrans)
  else:
    print(1,backtrans)
    # 다 펼쳐진 상태에서 모든 질문 가져오기
    page_nums=driver.find_elements("class name","a-card__main")
    print(len(page_nums))

except Exception as e:
  #print(e)
  page_nums=driver.find_elements("class name","a-card__main")
  print(len(page_nums))



1 10개 더보기
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
303


In [ ]:
# 링크 가져오기
links=[]
for i in page_nums:
  l = i.find_element('class name', 'questionListCard__titleLink').get_attribute('href')
  links.append(l)
links

['https://www.a-ha.io/questions/4caa93d020284467963f98f765fb6a3e?status=published&order=recent&aha_term=%EA%B3%B5%EB%AA%A8%EC%A3%BC',
 'https://www.a-ha.io/questions/49e8a356f2ca7b1f9fbef8702a2f88b8?status=published&order=recent&aha_term=%EA%B3%B5%EB%AA%A8%EC%A3%BC',
 'https://www.a-ha.io/questions/4b90c9675d2d8b5d9bef31fab8ccd1b3?status=published&order=recent&aha_term=%EA%B3%B5%EB%AA%A8%EC%A3%BC',
 'https://www.a-ha.io/questions/410419e61a306d8eb309d7a9d38b2687?status=published&order=recent&aha_term=%EA%B3%B5%EB%AA%A8%EC%A3%BC',
 'https://www.a-ha.io/questions/4a64480173c60678a7635e777a0f14ff?status=published&order=recent&aha_term=%EA%B3%B5%EB%AA%A8%EC%A3%BC',
 'https://www.a-ha.io/questions/4f5aa6b600036a2bba0cef3fd8226a00?status=published&order=recent&aha_term=%EA%B3%B5%EB%AA%A8%EC%A3%BC',
 'https://www.a-ha.io/questions/43ac839b7028bc6198de4b88da3d2a7e?status=published&order=recent&aha_term=%EA%B3%B5%EB%AA%A8%EC%A3%BC',
 'https://www.a-ha.io/questions/43de8d20edf36425a948e79c639dcc

질의응답 가져오기 (BeatifulSoup)

In [ ]:
aha=pd.DataFrame(columns=['question','answer'])

for l in links:
  answer_list=[]
  response=requests.get(l)
  soup=BeautifulSoup(response.text,'html.parser')
  q=soup.find_all('div',{'class':'editor__content'})
  try:
    question=q[0].text
    for i in range(1,len(q)):
      answer=q[i].text
      answer_list.append(answer)
    # 답변 갯수만큼 question 갯수 맞춰줌
    aha = pd.concat([aha, pd.DataFrame({'question': [question]*len(answer_list), 'answer': answer_list})])
  # 질문이 없다..?
  except:
    print(l)

# 인덱스 정리
aha=aha.reset_index(drop=True)
aha

,question,answer
0,"제가 요즘 공모주(주식)를 하고있는데배정받는 수량을 늘리기 위해서 가족들(형,부모님...","안녕하세요. 문용현 세무사입니다.차용증을 작성하셔서 계좌이체를 받으시고, 환불이 되..."
0,요즘 공모주를 주변에서 이야기 많이하네요. 400%까지도 오른다고 다들 사던데 정말...,안녕하세요. 김재철 경제·금융전문가입니다.최대수익400프로는 맞으나공모주도 옥석을 ...
1,요즘 공모주를 주변에서 이야기 많이하네요. 400%까지도 오른다고 다들 사던데 정말...,안녕하세요. 유동현 경제·금융전문가입니다.400%까지 올라갈 수도 있지만 오히려 공...
2,요즘 공모주를 주변에서 이야기 많이하네요. 400%까지도 오른다고 다들 사던데 정말...,안녕하세요. 최연준 경제·금융전문가입니다.공모주는 하루 400프로까지 오를수는 있으...
3,요즘 공모주를 주변에서 이야기 많이하네요. 400%까지도 오른다고 다들 사던데 정말...,안녕하세요. 전중진 경제·금융전문가입니다.공모주와 같은 경우 시세변동폭이 변화하여마...
...,...,...
2,부동산 아파트가 20억짜리 그리고 6억짜리(24억인데 가족끼리 N/1 지분 보유)를...,안녕하세요. 조장우 AFPK입니다.양도소득세가 클 때에는 보유하는게 더 낫기는 하지...
0,안녕하세요~아이 태어나던 해에 통장을 만들어 아이앞으로 받는돈 모아준게 800정도 ...,안녕하세요. 백승호 세무사입니다.현금증여의 경우 원칙적으로 입금일로부터 3개월 이내...
1,안녕하세요~아이 태어나던 해에 통장을 만들어 아이앞으로 받는돈 모아준게 800정도 ...,안녕하세요. 문용현세무사입니다.1. 이체기간별로 필터링하여 기간별 증여금액만 파악할...
0,3년차 직장인입니다.현재 월수입 세후 400만원 수준(간단한 부업이나 앱테크 등으로...,안녕하세요. 박지암 AFPK/보험전문가입니다.잘하고 계십니다. 월 고정지출에 개인연...


In [ ]:
aha.groupby('question').count()

,answer
question,
공모주 신청과 절차는 어떤식으로 진행되는지 알고싶습니다.예수금이란개념?? 신청을 하려면 금액을 어떤식으로 산정되서 주식의 개수를 받을수있는지금액적인 절차와 신청절차에 대해서 알고싶어요,2
"공모주에 참가한다고 하였을때, 금액을 이체시켜 몇주를 받는다고 가정하면 거래시작일에는 보통 입찰?가격보다 높게형성되나요?확률적으로아니라면 어떤걸 고려해야하나요",8
"안녕하세요. 저는 열심히 일하는 대한민국 직장인입니다. 올해 처음으로 종합소득세 신고하라고 연락을 받았습니다. 그런데 사업소득에 출처를 도무지 알수가 없네요주업종코드는 525101이라고 금액만 나와있어 어디서 수입이 발생된것인지 도무지 알수가 없네요ㅜㅜ작년 2022년 다양한 오픈마켓에서 중고나 새상품 되팔이 부업을 하였습니다. 그리고 가족명의로 주식, 공모주,P2P 등 금융관련 상품도 하였구요.사업소득이 어디서 수입으로 잡힌건지 알수있는 방법 없을까요? 세무서는 전화통화가 않되네요ㅜㅜ직장인이라 갈려니 힘들고 답답합니다.",3
"알멕이가 물멕였요 ㅠ.ㅠ 0.34주라서 못받을수도있겠구나했는데 정말로 못받았네요ㅠ.ㅠ 균등배정을 받고 못받고 혹시 기준이있나요? 예를들어 최소수량만 청약했을땐 확률이 적어진다거나 주식잔고가 얼마냐에따라(?) 그러진않을것같긴한데;; 암튼, 어떤 기준이 혹시나있을까싶어서 아쉬운마음에 질문드려요~",1
완전 주식 초초보입니다. 주식 공모주 청약에서 균등비례와 비례배정이 있는데요... 균등배정은 대충 알겠는데 비례배정은 정확히 모르겠습니다. 예로 경쟁률이 1000:1이 었다고 가정하면 균등배정과 비례배정은 어떻게 산출되나요? 추가로 대략 비례배정 1주라도 받으려면 최소증거금의 몇 배를 넣어야 하는것인지요? 돈 많은 사람이 수억.수십억 공모 쳥약 입금 하면 비례배정은 수백 수천 넣은 사람은 1주도 못받게되는거죠? 비례배정 1주를 받는 댸락의 산출 공식이 있나요?,2
...,...
현재 5세 10세 자녀 양육중인데 공모주 청약위해 증권 계좌를 많이 만들어 청약하고 있습니다 비례청약시 몇천만원씩 넣었다가 청약 끝나면 다시 제 계좌로 넣는데 이런 경우도 증여세 내야하나요,1
"현재 다니고 있는 회사 상장시 부모님으로 부터 금액 일부를 받아 공모주 매수를 하였습니다.1년뒤 거래 가능 시점 도래시에 해당금액만큼의 주식을 부모님께 양도하게 되는 경우 별도의 차용증 작성 및 공증이 필요한지 문의드립니다.정리하자면,1) 공모가 100원 인 경우 100주 체결 위해 10,000원 부모님께 계좌 송금받음2) 1년뒤 거래 가능 시점에 100주를 부모님 주식 계좌로 이체 예정3) 해당 계좌이체는 증여의 개념이 아님에 따라 별도 공증 서류가 필요한지 여부4) 해당 기간내에 주택 구매를 할 경우 증여로 판단 가능성이 있는지, 해당 경우 해당 입금은 공모주 거래를 위한 것임을 소명만 하면 되는건지 여부",2
현재 해군 복무중인 22살 대학생입니다. 지금 현재 적금 저축하고 남은돈 60에서 20은 배당주에 투자하고 있으며 20은 공모주 청약하는데 쓰고있으며 남은 20으로 생활비로 쓰고있는데 이렇게 투자 게속해도 되는걸까요?,4


In [ ]:
aha.to_csv('/content/drive/MyDrive/miraeasset/아하_펼친ver.csv', index=False)

답변 앞문장 2개 제거

In [ ]:
for i in range(len(aha['answer'])):
  # 인삿말 두 문장 제거
  if '안녕하' in aha['answer'][i]:
    ans = aha['answer'][i].split('.')[2:]
    res = ''
    for a in ans:
      res+=a+'. '
    aha['answer'][i]=res
  else:
    print(aha['answer'][i])
aha

차용증을 작성하셔서 계좌이체를 받으시고, 환불이 되어 다시 돌려주신다면 증여로 보지 않습니다. 도움이 되셨길 바랍니다.  감사합니다. . 
최대수익400프로는 맞으나공모주도 옥석을 가려서 투자하셔야해요묻지마투자는 리스크가커요. 
400%까지 올라갈 수도 있지만 오히려 공모가 아래로 떨어질 가능성도 있기 때문에 경쟁률이 적은 공모주는 위험할 수 있으니 잘 알아보시고 하시길 바랍니다. 감사합니다. . 
공모주는 하루 400프로까지 오를수는 있으나아직 그정도로 오른 공모주는 없었습니다상장일에 공모가보다 내려갈수도있기에투자에 유의하셔야합니다인기가 좋은 종목은균등1주받으려면 100만원 정도는 필요하고비례1주 받으려면 몇천까지 필요합니다공모 한번하는데 수수료는 2~3천원 합니다참고하시기 바랍니다  . 
공모주와 같은 경우 시세변동폭이 변화하여마이너스 60퍼센트에서 플러스 400퍼센트까지시세를 볼 수 있습니다. 마이너스도 충분히 가능하니 참고하시길 바랍니다.  . 
공모주의 시초가 매도는 공모주의 시초 가격보다 낮은 가격으로 매도를 걸어두시면 하실 수 있습니다.  공모주는 공모가격의 60%~400% 사이에서 시장가격이 결정되는데 60%가격으로 시작하는 것 까지 고려하여 매도가격을 설정해두시면 시초가 매도를 하실 수 있습니다. 예를 들어서 공모가격이 10,000원인 경우에는 6천원에 시초가 매도를 설정해두시면 무조건적인 매도가 체결될 수 있습니다.  . 
공모가 대비하여 상당히 낮은 가격에 매도를 걸어놓으시면 시초가에 매도가 가능할 것이니참고하시길 바랍니다.  . 
공모주 일정을 보면 청약을 주간(담당)하는 증권사가 있습니다.  해당 증권사 계좌를 가져야만 청약이 가능하기 때문에 해당 증권사 계좌가 없다면 신규로 개설하셔야 합니다. . 
공모주를 청약하기 위해서는 청약을 진행하는 주관사의 주식계좌를 가지고 있어야 합니다.  예를 들어 삼성증권에서 진행하는 공모주는 삼성증권 계좌가 있어야 합니다. . 
보통 증권사마다 공모주 청약 등의 카테고리가따로 설정되어 있고 해당 카테고

,question,answer
0,"제가 요즘 공모주(주식)를 하고있는데배정받는 수량을 늘리기 위해서 가족들(형,부모님...","차용증을 작성하셔서 계좌이체를 받으시고, 환불이 되어 다시 돌려주신다면 증여로 보지..."
1,요즘 공모주를 주변에서 이야기 많이하네요. 400%까지도 오른다고 다들 사던데 정말...,최대수익400프로는 맞으나공모주도 옥석을 가려서 투자하셔야해요묻지마투자는 리스크가커요.
2,요즘 공모주를 주변에서 이야기 많이하네요. 400%까지도 오른다고 다들 사던데 정말...,400%까지 올라갈 수도 있지만 오히려 공모가 아래로 떨어질 가능성도 있기 때문에 ...
3,요즘 공모주를 주변에서 이야기 많이하네요. 400%까지도 오른다고 다들 사던데 정말...,공모주는 하루 400프로까지 오를수는 있으나아직 그정도로 오른 공모주는 없었습니다상...
4,요즘 공모주를 주변에서 이야기 많이하네요. 400%까지도 오른다고 다들 사던데 정말...,공모주와 같은 경우 시세변동폭이 변화하여마이너스 60퍼센트에서 플러스 400퍼센트까...
...,...,...
872,부동산 아파트가 20억짜리 그리고 6억짜리(24억인데 가족끼리 N/1 지분 보유)를...,양도소득세가 클 때에는 보유하는게 더 낫기는 하지만 확실히 자산 규모가 큰 만큼 종...
873,안녕하세요~아이 태어나던 해에 통장을 만들어 아이앞으로 받는돈 모아준게 800정도 ...,현금증여의 경우 원칙적으로 입금일로부터 3개월 이내에 증여세 신고를 하면 되는 것이...
874,안녕하세요~아이 태어나던 해에 통장을 만들어 아이앞으로 받는돈 모아준게 800정도 ...,1. 이체기간별로 필터링하여 기간별 증여금액만 파악할수 있게 캡쳐자료 등을 첨부하...
875,3년차 직장인입니다.현재 월수입 세후 400만원 수준(간단한 부업이나 앱테크 등으로...,잘하고 계십니다. 월 고정지출에 개인연금이 포함된 것 같은데 지출보다는 저축에 들...


In [ ]:
aha.to_csv('/content/drive/MyDrive/miraeasset/아하_filtered.csv', index=False)